In [2]:
%load_ext autoreload
%autoreload 2

In [15]:
import pandas as pd

from datetime import datetime, timedelta

dt = datetime.utcnow()

In [22]:
df1 = pd.DataFrame(
    {
        "a": [1, 2, 3],
        "b": [1.0, 2.0, 3.0],
        "c": ["A", "B", "C"],
        "d": [dt, dt + timedelta(1), dt + timedelta(2)],
    },
    index=["0", "1", "2"],
)

df2 = pd.DataFrame({"Z": [3, 2, 1], "a": [4, 5, 6]}, index=["0", "1", "2"])
es_index = "test3"

In [23]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

In [24]:
import eland as ed

In [25]:
ed.pandas_to_eland(df1, es_client, es_index, es_dropna=True, 
                               es_if_exists='append', use_pandas_index_for_es_ids=False)

/Users/sidhuas/.pyenv/versions/3.9.1/envs/eland/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


,a,b,c,d


In [26]:
ed.pandas_to_eland(df2, es_client, es_index, es_dropna=True, enforce_index_schema=False,
                               es_if_exists='append', use_pandas_index_for_es_ids=False)

/Users/sidhuas/.pyenv/versions/3.9.1/envs/eland/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


,Z,a,b,c,d
6KaKA34BeVG4RuLOG_yX,NaN,1,1.0,A,2021-12-29 00:13:36.956031
6aaKA34BeVG4RuLOG_yX,NaN,2,2.0,B,2021-12-30 00:13:36.956031
6qaKA34BeVG4RuLOG_yX,NaN,3,3.0,C,2021-12-31 00:13:36.956031


In [27]:
ed.DataFrame(es_client, es_index)

/Users/sidhuas/.pyenv/versions/3.9.1/envs/eland/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


,Z,a,b,c,d
6KaKA34BeVG4RuLOG_yX,NaN,1,1.0,A,2021-12-29 00:13:36.956031
6aaKA34BeVG4RuLOG_yX,NaN,2,2.0,B,2021-12-30 00:13:36.956031
6qaKA34BeVG4RuLOG_yX,NaN,3,3.0,C,2021-12-31 00:13:36.956031
66aKA34BeVG4RuLOH_xK,3.0,4,NaN,NaN,NaT
7KaKA34BeVG4RuLOH_xK,2.0,5,NaN,NaN,NaT
7aaKA34BeVG4RuLOH_xK,1.0,6,NaN,NaN,NaT
